In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator

# Import Dataset

In [ ]:
IMG_SIZE=256
BATCH_SIZE=32

In [ ]:
def import_dataset(path, IMG_SIZE=256,BATCH_SIZE=32):
  dataset=tf.keras.preprocessing.image_dataset_from_directory(
      # path,
      shuffle=True,
      image_size=(IMG_SIZE,IMG_SIZE),
      batch_size=BATCH_SIZE
  )
  return dataset

In [ ]:
dataset=import_dataset(path,IMG_SIZE,BATCH_SIZE)

# Visualization

In [ ]:
class_names=dataset.class_names

In [ ]:
for image, label in dataset.table(1):
  print(image.shape)
  print(label.numpy())

In [ ]:
def visualize_data(dataset):
  plt.figure(figsize=(10,10))
  for image, label in dataset.table(1):
    for i in range(10):
      ax=plt.subplot(2,5,i+1)
      plt.imshow(image[i].numpy().astype("uint8"))
      plt.title(class_names[label[i]])
      plt.axis("off")

In [ ]:
visualize_data(dataset)

# Split the Data

In [ ]:
def train_test_split(dataset,train_split=0.8 , test_split=0.1, validation_split=0.1 ,shuffle=True,shuffle_size=10000):
  dataset_size=len(dataset)
  if shuffle:
    datatset=dataset.shuffle(shuffle_size,seed=10)

  train_size=int(dataset_size*train_split)
  val_size=int(validation_split*dataset_size)

  train_ds=dataset.take(train_size)
  val_ds=dataset.skip(train_size).take(val_size)
  test_ds=dataset.skip(train_size).skip(val_size)

  return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds=train_test_split(dataset)

- Check Class imbalance in train, valid and test

# Optimize the Data


In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

# Scale Data

In [ ]:
resize_and_scale=tf.keras.Sequential([
          layers.experimental.preprocessing.Resizing(IMG_SIZE,IMG_SIZE),
          layers.experimental.preprocessing.Rescaling(1/255)

])

# Data Augmentation

In [ ]:
train_dataset_gen =ImageDataGenerator(
    preprocessing_function=resize_and_scale,
    rotation_range=60,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.0, 0.8)
)

train_generator = train_dataset_gen.flow_from_directory(
        'dataset/train',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=32,
        class_mode="sparse",
#         save_to_dir="C:\\Code\\potato-disease-classification\\training\\AugmentedImages"
)

In [ ]:
class_names = list(train_generator.class_indices.keys())
class_names

In [ ]:
val_dataset_gen =ImageDataGenerator(
    preprocessing_function=resize_and_scale,
    rotation_range=60,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.0, 0.8)
)

validation_generator = val_dataset_gen.flow_from_directory(
        'dataset/val',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=32,
        class_mode="sparse"
)

In [ ]:
test_dataset_gen =ImageDataGenerator(
    preprocessing_function=resize_and_scale,
    rotation_range=60,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.0, 0.8)
)

test_generator = test_dataset_gen.flow_from_directory(
        'dataset/test',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=32,
        class_mode="sparse"
)

# Model Building

In [ ]:
CHANNELS=3
input_shape = (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS)

classifier_model1 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/rishit-dagli/plant-disease/1", input_shape=input_shape, trainable=True)
])

classifier_model2 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/agripredict/disease-classification/1", input_shape=input_shape, trainable=True)
])

classifier_model3 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/aiy/vision/classifier/plants_V1/1", input_shape=input_shape, trainable=True)
])

In [ ]:
NUM_CLASSES=???

In [ ]:
# change the model used here to classifier_model2 or classifier_model3 to use the other models

model_used = classifier_model1

model = models.Sequential([
    model_used,
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])
model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=30, verbose=1, batch_size=BATCH_SIZE)

In [ ]:
scores = model.evaluate(test_ds)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
EPOCHS = 30

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
import numpy as np

for image_batch, label_batch in test_generator:
    first_image = image_batch[0]
    first_label = int(label_batch[0])
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(image_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])
    
    break


In [ ]:
model.save("../model_name.h5")